# Libraries

In [1]:
!pip install scikit-learn

In [2]:
import os
import pickle

import wandb
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Initialization

In [3]:
# Initialize a WandB Run
wandb.init(project="First", job_type="log_data")

# Log the `data` directory as an artifact
artifact = wandb.Artifact('Titanic', type='dataset', metadata={"Source": "https://www.kaggle.com/competitions/titanic/data"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

# End the WandB Run
wandb.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: nyamadie1 (nyamadie1-whisper-health). Use `wandb login --relogin` to force relogin


wandb: Adding directory to artifact (./data)... Done. 0.0s


In [11]:
# Initialize a WandB Run
run = wandb.init(project="First", job_type="log_data")

# Fetch the dataset artifact 
artifact = wandb.use_artifact('nyamadie1-whisper-health/First/Titanic:v0', type='dataset')
artifact_dir = artifact.download()

wandb:   2 of 2 files downloaded.  


In [21]:
import wandb
run = wandb.init(project="First", job_type="log_data")

artifact = run.use_artifact('nyamadie1-whisper-health/First/Titanic:v1', type='dataset')
artifact_dir = artifact.download()

wandb:   2 of 2 files downloaded.  


In [12]:
train_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))

In [13]:
num_train_examples = int(0.8 * len(train_df))
num_val_examples = len(train_df) - num_train_examples

print(num_train_examples, num_val_examples)

712 179


In [14]:
train_df["Split"] = ["Train"] * num_train_examples + ["Validation"] * num_val_examples
train_df.to_csv("data/train.csv", encoding='utf-8', index=False)

In [15]:
# Log the `data` directory as an artifact
artifact = wandb.Artifact('Titanic', type='dataset', metadata={"Source": "https://www.kaggle.com/competitions/titanic/data"})
artifact.add_dir('data')
wandb.log_artifact(artifact)

# End the WandB Run
wandb.finish()

wandb: Adding directory to artifact (./data)... Done. 0.0s


# Add Table

In [19]:
import wandb
import os
import pandas as pd

# Initialize a WandB Run
run = wandb.init(project="First", job_type="explore_data")

# Fetch the latest version of the dataset artifact 
artifact = run.use_artifact('nyamadie1-whisper-health/First/Titanic:latest', type='dataset')
artifact_dir = artifact.download()

# Read the files
train_val_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))


wandb:   2 of 2 files downloaded.  


In [20]:
# Create tables corresponding to datasets
train_val_table = wandb.Table(dataframe=train_val_df)
test_table = wandb.Table(dataframe=test_df)

# Log the tables to Weights & Biases
run.log({
    "Train-Val-Table": train_val_table,
    "Test-Table": test_table
})

# End the W&B Run
#run.finish()
# End the WandB Run
wandb.finish()

# Fit Model

In [33]:
# Initialize a WandB Run
wandb.init(project="First", name="baseline_experiment-2", job_type="train")

# Fetch the latest version of the dataset artifact 
artifact = wandb.use_artifact('nyamadie1-whisper-health/First/Titanic:latest', type='dataset')
artifact_dir = artifact.download()

# Read the files
train_val_df = pd.read_csv(os.path.join(artifact_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(artifact_dir, "test.csv"))

wandb:   2 of 2 files downloaded.  


In [34]:
features = ["Pclass", "Sex", "SibSp", "Parch"]
X_train = pd.get_dummies(train_val_df[features][train_val_df["Split"] == "Train"])
X_val = pd.get_dummies(train_val_df[features][train_val_df["Split"] == "Validation"])
y_train = train_val_df["Survived"][train_val_df["Split"] == "Train"]
y_val = train_val_df["Survived"][train_val_df["Split"] == "Validation"]

In [35]:
model_params = {"n_estimators": 100, "max_depth": 15, "random_state": 1}
wandb.config = model_params

model = RandomForestClassifier(**model_params)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_probas_train = model.predict_proba(X_train)
y_pred_val = model.predict(X_val)
y_probas_val = model.predict_proba(X_val)

In [36]:
wandb.log({
    "Train/Accuracy": accuracy_score(y_train, y_pred_train),
    "Validation/Accuracy": accuracy_score(y_val, y_pred_val),
    "Train/Presicion": precision_score(y_train, y_pred_train),
    "Validation/Presicion": precision_score(y_val, y_pred_val),
    "Train/Recall": recall_score(y_train, y_pred_train),
    "Validation/Recall": recall_score(y_val, y_pred_val),
    "Train/F1-Score": f1_score(y_train, y_pred_train),
    "Validation/F1-Score": f1_score(y_val, y_pred_val),
})

In [37]:
label_names = ["Not-Survived", "Survived"]

wandb.sklearn.plot_class_proportions(y_train, y_val, label_names)
wandb.sklearn.plot_summary_metrics(model, X_train, y_train, X_val, y_val)
wandb.sklearn.plot_roc(y_val, y_probas_val, labels=label_names)
wandb.sklearn.plot_precision_recall(y_val, y_probas_val, labels=label_names)
wandb.sklearn.plot_confusion_matrix(y_val, y_pred_val, labels=label_names)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [38]:
# Save your model
with open("random_forest_classifier.pkl", "wb") as f:
    pickle.dump(model, f)

# Log your model as a versioned file to Weights & Biases Artifact
artifact = wandb.Artifact(f"titanic-random-forest-model", type="model")
artifact.add_file("random_forest_classifier.pkl")
wandb.log_artifact(artifact)


# End the WandB Run
wandb.finish()

Train/Accuracy,▁
Train/F1-Score,▁
Train/Presicion,▁
Train/Recall,▁
Validation/Accuracy,▁
Validation/F1-Score,▁
Validation/Presicion,▁
Validation/Recall,▁
Train/Accuracy,0.8118
Train/F1-Score,0.73307
Train/Presicion,0.82143


In [ ]:
hnbh